In [3]:
''' Libraries ''' 
import requests
import os
import pandas as pd

from urllib.request import urlopen
import functools as ft


In [4]:
def tag(nombre, api):
    name = nombre + '_'+api
    return name
tabla = pd.read_excel(r'../Merge.xlsx')
#tabla.drop(columns='Unnamed: 0', inplace=True)
# ALPHA Vantage
columnas_Alpha  =['fiscalDateEnding', 'reportedCurrency']+ tabla.tag_alpha.tolist()
tabla.head()

,tag_tiingo,desc_tiingo,tag_alpha,desc_alpha,tag_finmodprep,tag_finn,tag_eod,tag_sharadar
0,totalAssets,Total Assets,totalAssets,"Assets, Total",totalAssets,totalAssets,totalAssets,ASSETS
1,totalLiabilities,Total Liabilities,totalLiabilities,"Liabilities, Total",totalLiabilities,totalLiabilities,totalLiab,LIABILITIES
2,sga,"Selling, General & Administrative",sellingGeneralAndAdministrative,"Selling, General and Administrative Expense, T...",sellingGeneralAndAdministrativeExpenses,sgaExpense,sellingGeneralAdministrative,SGNA
3,rnd,Research & Development,researchAndDevelopment,"Research and Development Expense, Total",researchAndDevelopmentExpenses,researchDevelopment,researchDevelopment,RND
4,ebitda,EBITDA,ebitda,"Earnings Before Interest, Taxes, Depreciation,...",ebitda,ebit,ebitda,EBITDA


In [4]:
def alpha_vantage(ticker, period='annualReports'):
    ''' Libraries ''' 
    import requests
    import os
    import pandas as pd

    from urllib.request import urlopen
    import functools as ft

    ''' taggin function'''
    def tag(nombre, api):
        name = nombre + '_'+api
        return name
        

    ''' Request'''
    url = f'https://www.alphavantage.co/query?function=Cash_flow&symbol={ticker}&apikey=6DWJBYP02X4S13GL'
    r = requests.get(url)
    data = r.json()
    df_cash = pd.DataFrame(data[period])
    

    url = f'https://www.alphavantage.co/query?function=Balance_Sheet&symbol={ticker}&apikey=6DWJBYP02X4S13GL'
    r = requests.get(url)
    data = r.json()
    df_balance = pd.DataFrame(data[period])

    url = f'https://www.alphavantage.co/query?function=Income_Statement&symbol={ticker}&apikey=6DWJBYP02X4S13GL'
    r = requests.get(url)
    data = r.json()
    df_income = pd.DataFrame(data[period])

    dfs = [ df_cash, df_balance , df_income]
    df_alpha = ft.reduce(lambda left, right: pd.merge(left, right, on='fiscalDateEnding'),dfs )
    
    tabla = pd.read_excel(r'../Merge.xlsx')
    columnas_Alpha = ['fiscalDateEnding'] + tabla.tag_alpha.tolist()
    
    df_alpha = df_alpha[columnas_Alpha]
    df_alpha = df_alpha.loc[:, ~df_alpha.columns.duplicated()]

    dict_alpha = {}
    
    '''
    for i in range(len(df_alpha.columns)):
        dict_alpha[df_alpha.columns[i]] = tag(df_alpha.columns[i],'alpha' )
    df_alpha.rename(columns=dict_alpha, inplace=True)
    ''' 
    df_alpha.insert(0,'ticker' , f'{ticker}')
    df_alpha.fiscalDateEnding = pd.to_datetime(df_alpha.fiscalDateEnding)
    df_alpha.insert(1, 'date',df_alpha.fiscalDateEnding.dt.strftime('%Y-%m'))
    
    return df_alpha


In [5]:
alpha_vantage('IBM')


,ticker,date,fiscalDateEnding,totalAssets,totalLiabilities,sellingGeneralAndAdministrative,researchAndDevelopment,ebitda,deferredRevenue,costOfRevenue,...,inventory,operatingIncome,comprehensiveIncomeNetOfTax,currentAccountsPayable,cashAndShortTermInvestments,commonStock,currentNetReceivables,capitalExpenditures,totalShareholderEquity,changeInOperatingAssets
0,IBM,2021-12,2021-12-31,132001000000,113005000000,18745000000,6488000000,9551000000,16095000000,25865000000,...,1649000000,4786000000,11299000000,3955000000,6650000000,57319000000,14977000000,2062000000,18901000000,-1510000000
1,IBM,2020-12,2020-12-31,155971000000,135244000000,20561000000,6262000000,7986000000,17134000000,24314000000,...,1812000000,4609000000,4850000000,4033000000,13812000000,56556000000,18738000000,2618000000,20597000000,-5088000000
2,IBM,2019-12,2019-12-31,152186000000,131202000000,18724000000,5910000000,12685000000,15877000000,26181000000,...,1619000000,9004000000,10324000000,4896000000,8868000000,55895000000,23795000000,2286000000,20841000000,-569000000
3,IBM,2018-12,2018-12-31,123382000000,106452000000,19366000000,5379000000,13423000000,14610000000,42655000000,...,1682000000,10838000000,8252000000,6558000000,11997000000,55151000000,30563000000,3395000000,16796000000,-879000000
4,IBM,2017-12,2017-12-31,125356000000,107631000000,19680000000,5590000000,13530000000,15298000000,42196000000,...,1583000000,8813000000,8559000000,6451000000,12580000000,54566000000,31630000000,3229000000,17594000000,-1315000000
